In [9]:
from postgres_connect import *
path = '/Users/apple/Documents/tunnel-ssh.cer'

In [10]:
conn = get_conn('Yes',path)
query_path = 'rpr.sql'
with open(query_path, 'r') as file:
    custom_query = file.read()
df_report = pd.read_sql(custom_query,conn)
conn.close()

Tunnel Started
Connection Made


In [11]:
df_report

,shipping_city,hub,rider_id,rider_name,load,delivery_per,fasr,prepaid_fasr,cod_fasr,failed_collection_pendency,cod_collection_pendency,mtd_fake,pip_result
0,Bangalore,BLR FRH ECTY,Ningayya_10,Ningayya,41,0.0,0.0,0.0,0.0,0,1439.0,NaN,None
1,Bangalore,BLR FRH ECTY,Santhu_7022177696_010,Santhu,46,24.0,21.0,27.0,8.0,0,0.0,NaN,None
2,Bangalore,BLR FRH ECTY,Nithin_8951194124_010,Nithin,39,67.0,67.0,74.0,54.0,19,3578.0,NaN,None
3,Bangalore,BLR FRH ECTY,Charan_9901944827_010,Charan Kumar S,50,74.0,73.0,84.0,36.0,0,14424.0,NaN,None
4,Bangalore,BLR FRH ECTY,Vishnu_Prasad__10,Vishnu Prasad,5,60.0,75.0,75.0,NaN,0,29030.0,7.0,None
...,...,...,...,...,...,...,...,...,...,...,...,...,...
549,NCR,Noida-Franchise,Vikas_singh_20,Vikas singh,27,81.0,91.0,100.0,33.0,2,499.0,NaN,None
550,NCR,Noida-Franchise,RAM_9528858863_020,RAM KUMAR,33,91.0,93.0,89.0,100.0,0,10125.0,NaN,None
551,NCR,Noida-Franchise,Sandeep_Kumar__20,Sandeep Kumar,30,93.0,96.0,100.0,75.0,6,0.0,2.0,None
552,NCR,Noida-Franchise,sushant__20,sushant,22,100.0,100.0,100.0,100.0,3,0.0,NaN,None


In [12]:
# 1. Rename columns
df_report.rename(columns={
    'shipping_city': 'City',
    'hub': 'Hub',
    'rider_id': 'Rider ID',
    'rider_name': 'Name',
    'load': 'Load',
    'delivery_per': 'Del%',
    'fasr': 'FASR',
    'prepaid_fasr': 'Prepaid FASR',
    'cod_fasr': 'COD FASR',
    'failed_collection_pendency': 'Failed Pending',
    'cod_collection_pendency': 'COD Pending',
    'mtd_fake': 'MTD Fake',
    'pip_result': 'PIP Status'
}, inplace=True)

# 2. Remove NaN and None, make them blank
df_report=df_report.apply(lambda x: x.fillna(''))


In [13]:
city_dict1 = {
    'Bangalore':['mayank@blitznow.in','anil@blitznow.in','ma.saifullah@blitznow.in','syed.afrid@blitznow.in','dilip.kumar@blitznow.in','biswojit.jena@blitznow.in'],
    'Delhi':['mayank@blitznow.in','sanjay@blitznow.in','ajay.verma@blitznow.in','rahul.lochav@blitznow.in'],
    'Mumbai':['mayank@blitznow.in','anil@blitznow.in','david.kumar@blitznow.in','sumit.kamble@blitznow.in'],
    'Hyderabad':['mayank@blitznow.in','anil@blitznow.in','shankar.poddar@blitznow.in','chandrakanth.sunkoji@blitznow.in'],
    'Jaipur':['mayank@blitznow.in','sourabh.vijayvergia@blitznow.in','sourabh.vijay@blitznow.in','sanjay@blitznow.in']
}

city_dict2 = {
    'Bangalore': ['akshay@blitznow.in','akshay@growsimplee.com'],
}

In [14]:
import smtplib
from email.mime.text import MIMEText
from email.mime.multipart import MIMEMultipart
from email.mime.application import MIMEApplication

# Your email credentials
sender_email = "akshay@blitznow.in"
sender_password = "Outofblitz1!"

# Function to send emails
def send_email(to_email_list, subject, body, attachment_paths):
    msg = MIMEMultipart()
    msg['From'] = sender_email
    msg['To'] = ", ".join(to_email_list)
    msg['Subject'] = subject

    # Attach body
    msg.attach(MIMEText(body, 'html'))

    # Attach CSV files with updated names
    for attachment_path in attachment_paths:
        with open(attachment_path['path'], "rb") as file:
            part = MIMEApplication(file.read(), Name=f"{attachment_path['name']}_hub_report.csv")
            part['Content-Disposition'] = f'attachment; filename="{attachment_path["name"]}_hub_report.csv"'
            msg.attach(part)

    # Establish a connection to the SMTP server
    with smtplib.SMTP("smtp.gmail.com", 587) as server:
        server.starttls()
        server.login(sender_email, sender_password)
        server.sendmail(sender_email, to_email_list, msg.as_string())

# Iterate through city_dict
for city, email_list in city_dict2.items():
    # Initialize the HTML body for the email
    email_body = f"""
    <html>
      <head>
        <style>
          table {{
            border-collapse: collapse;
            width: 100%;
          }}
          th, td {{
            padding: 8px;
            text-align: center;
          }}
          th {{
            background-color: lightblue;
          }}
        </style>
      </head>
      <body>
        <h2>Hub-wise Report for {city}</h2>
    """

    # List to store paths of CSV files and their respective names
    csv_attachments = []

    # Iterate through each hub in the city
    for hub in df_report[df_report['City'] == city]['Hub'].unique():
        print(f"printing for {city} - {hub}")
        # Extract data for the current hub
        hub_data = df_report[(df_report['City'] == city) & (df_report['Hub'] == hub)].drop(columns=['City'])

        # Generate HTML table from DataFrame
        html_table = hub_data.to_html(index=False, escape=False)  # Added escape=False to prevent HTML escaping

        # Append the HTML table to the email body
        email_body += f"<h3>Hub: {hub}</h3>{html_table}"

        # Save DataFrame to CSV file
        csv_path = f"{city}_{hub}_hub_report.csv"
        hub_data.to_csv(csv_path, index=False)

        # Append CSV path and name to the list
        csv_attachments.append({'path': csv_path, 'name': f"{city}_{hub}"})

    # Close the HTML body
    email_body += """
      </body>
    </html>
    """

    # Send email to the city's email list with all CSV files attached
    send_email(email_list, f"Rider Performance Report for {city}", email_body, csv_attachments)


printing for Bangalore - BLR FRH ECTY
printing for Bangalore - DS BLR BOMM
printing for Bangalore - DS BLR HBBL
printing for Bangalore - DS BLR MTH
printing for Delhi - DEL FRH GKP
printing for Delhi - DS DEL DWK
printing for Delhi - DS DEL HKMN
printing for Delhi - DS DEL RNGR
printing for Delhi - GGKC-Franchise
printing for Delhi - ONFC-Franchise
printing for Delhi - RHNP-Franchise
printing for Mumbai - DS BOM KRL
printing for Mumbai - DS BOM SWRI
printing for Mumbai - DS BOM THAN
printing for Mumbai - GRGN-Franchise
printing for Hyderabad - DS HYD KMP
printing for Hyderabad - DS HYD SRNG
printing for Hyderabad - DS HYD TRNK
printing for Jaipur - DS JPR NPRG


2024-01-27 12:09:58,884| ERROR   | Socket exception: Operation timed out (60)
2024-01-27 12:09:58,917| ERROR   | Socket exception: Operation timed out (60)
